# Imputing missing values using LinearRegression

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('house_prices_multivariate.csv')
df.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold,SalePrice
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2008,208500
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,2007,181500
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,2008,223500
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2006,140000
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,2008,250000


# Check for missing values

In [3]:
df.isnull().values.any()

False

# Feature names

In [4]:
df.columns.values

array(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
       'PoolArea', 'MiscVal', 'YrSold', 'SalePrice'], dtype=object)

# Function to get the most correlated feature pairs

In [5]:
## Functions to get top correlated feature pairs 
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

# credits : Arun (stackoverflow : https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas)

In [6]:
get_top_abs_correlations(df)

GarageCars   GarageArea      0.831481
YearBuilt    GarageYrBlt     0.825667
TotalBsmtSF  1stFlrSF        0.822469
GrLivArea    TotRmsAbvGrd    0.820975
OverallQual  SalePrice       0.787228
dtype: float64

# Replace some values in '1stFlrSF' to 'NaN' to impute later

In [7]:
# Before that ! save the originals and their positions, given the condition.
poz = []
y_orig = []
for m, val2 in enumerate(df['1stFlrSF']):
    if val2 >= 1300:
        y_orig.append(df['1stFlrSF'][m])
        poz.append(m)

In [8]:
df['1stFlrSF'] = df['1stFlrSF'].map(lambda x: np.nan if x >= 1300 else x).values
print df['1stFlrSF']

0        856.0
1       1262.0
2        920.0
3        961.0
4       1145.0
5        796.0
6          NaN
7       1107.0
8       1022.0
9       1077.0
10      1040.0
11      1182.0
12       912.0
13         NaN
14      1253.0
15       854.0
16      1004.0
17      1296.0
18      1114.0
19         NaN
20      1158.0
21      1108.0
22         NaN
23      1060.0
24      1060.0
25         NaN
26       900.0
27         NaN
28         NaN
29       520.0
         ...  
1349    1048.0
1350     804.0
1351       NaN
1352     734.0
1353     958.0
1354     968.0
1355     962.0
1356    1126.0
1357       NaN
1358     864.0
1359       NaN
1360    1236.0
1361    1040.0
1362       NaN
1363     848.0
1364    1026.0
1365     952.0
1366       NaN
1367     913.0
1368    1188.0
1369    1220.0
1370     796.0
1371       NaN
1372    1072.0
1373    1221.0
1374     953.0
1375       NaN
1376    1188.0
1377    1078.0
1378    1256.0
Name: 1stFlrSF, Length: 1379, dtype: float64


# Check for missing values again

In [9]:
df.isnull().values.any()

True

# Make 4 copies of the dataframe inorder to check r2_score after imputing by 
* Mean -> df1
* Median -> df2
* Mode -> df3
* Regression -> df4

In [10]:
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()

# Imputing with mean, median and mode

In [11]:
df1['1stFlrSF'] = df1['1stFlrSF'].fillna(df1['1stFlrSF'].mean())
df2['1stFlrSF'] = df2['1stFlrSF'].fillna(df2['1stFlrSF'].median())
df3['1stFlrSF'] = df3['1stFlrSF'].fillna(df3['1stFlrSF'].mode()[0])

In [12]:
# Save the values of the imputed to lists to compare
def imputed_list(DF):
    lis = []
    for i in poz:
            lis.append(DF['1stFlrSF'][i])
    return lis
y_mean = imputed_list(df1)
y_median = imputed_list(df2)
y_mode = imputed_list(df3)

# Function to predict and impute missing values 

In [13]:
def regression_imputer(col1,col2):
    # import the necessary libraries
    import pandas as pd
    # using below assignment to curb 'SettingWithCopyWarning'
    pd.options.mode.chained_assignment = None
    import numpy as np
    from sklearn.linear_model import LinearRegression
    
    # make a dataframe out of the columns
    df = pd.DataFrame({'imputer':col1,'imputee':col2})
    
    # check if the imputee has missing values. return if no
    if not df['imputee'].isnull().values.any():
        return df['imputee']
    
    # train and test split
    X_train = df['imputer'].where(df['imputee'].notnull()).dropna()
    y_train = df['imputee'].where(df['imputee'].notnull()).dropna()
    X_test = df['imputer'].where(df['imputee'].isnull()).dropna()
    
    # train model and return predicted values
    model = LinearRegression()
    model = model.fit(X_train[:,np.newaxis],y_train)
    y_predict = model.predict(X_test[:,np.newaxis])
    
    # update the missing values with predicted values
    pos = 0
    for i,val in enumerate(df['imputee']):
        if pd.isnull(val):
            df['imputee'][i] = y_predict[pos]
            pos += 1
    return df['imputee']

## The 2 correlated features, (one with all values and other with missing values)
## to be fed as input to our function as an array

In [14]:
col1 = df4['TotalBsmtSF']
col2 = df4['1stFlrSF']

# Function call to impute the column by regression

In [15]:
df4['1stFlrSF'] = regression_imputer(col1,col2)
# df4['1stFlrSF'].isnull().values.any()
# df4['1stFlrSF']

In [16]:
# Save the values of the imputed to lists to compare
y_prediction = imputed_list(df4)
print len(y_prediction), len(y_orig), len(y_mean), len(y_median), len(y_mode)

448 448 448 448 448



# Check the r2 score for each type of imputation

In [17]:
# list for the sake of printing
method = ['mean','median','mode','regression']

# Call in train_test_split
from sklearn.model_selection import train_test_split

# Call in the metric
from sklearn.metrics import r2_score

# Create a global model
from sklearn.linear_model import LinearRegression
model = LinearRegression()

def test_imputation(lis):
    pos = 0
    for dataframe in lis:
        X = dataframe.iloc[:,0:-1]
        y = dataframe.iloc[:,-1]
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=9)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        r2Score = r2_score(y_pred,y_test)
        print " r2score of",method[pos],"\t:",r2Score
        pos += 1

In [18]:
# Let's pass a list of df's
lis = [df1,df2,df3,df4]

test_imputation(lis)

 r2score of mean 	: 0.692974802164
 r2score of median 	: 0.692975151348
 r2score of mode 	: 0.693052480887
 r2score of regression 	: 0.692892534023


# Got lesser score than the rest !!! :/ Let's blame the dataset, eh ? -_-

# Comparing the values below

In [20]:
compare = pd.DataFrame({'y_orig':y_orig,'y_prediction':y_prediction,'y_mean':y_mean,'y_median':y_median,'y_mode':y_mode},index=poz)
compare.head()

,y_mean,y_median,y_mode,y_orig,y_prediction
6,962.433942,961.0,864.0,1694,1345.670401
13,962.433942,961.0,864.0,1494,1254.488311
19,962.433942,961.0,864.0,1339,1033.656687
22,962.433942,961.0,864.0,1795,1388.886913
25,962.433942,961.0,864.0,1600,1288.681595


# The End